In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/almost/almost_done_df


In [2]:
df=pd.read_csv('/kaggle/input/almost/almost_done_df')
df.drop(['Course Number','Prerequisites_List','Unnamed: 0','Year'],axis=1,inplace=True)

In [3]:
df

,EX,A,B,C,D,P,F,Semester,name,L-T-P,Department Code
0,0,0,0,4,0,0,0,Spring,ENVIRONMENTAL SOCIOLOGY,3,HS
1,4,12,14,16,10,2,0,Spring,MANUFACTURING MANAGEMENT,3,EP
2,14,38,30,4,0,0,0,Spring,AUDIO SYSTEMS ENGINEERING,3,ET
3,3,3,13,6,2,0,0,Spring,GENERAL PSYCHOLOGY,4,HS
4,9,9,3,2,1,0,0,Spring,GERMAN,3,HS
...,...,...,...,...,...,...,...,...,...,...,...
183,2,2,0,0,0,0,1,Spring,STATISTICAL INFERENCE,4,MA
184,0,0,0,0,0,1,0,Autumn,MOLECULAR SPECTROSCOPY & MOLECULAR STRUCTURE,4,CY
185,0,1,0,0,0,0,0,Autumn,METAL COMPLEXES IN CATALYSIS & MATERIAL SCIENCE,3,CY
186,0,0,0,1,0,0,0,Autumn,STRUCTURE ANALYSIS BY SPECTROSCOPIC & CRYSTALL...,4,CY


In [4]:
df['name'].nunique()

82

In [5]:
# Group by course name and sum the corresponding grade columns
df = df.groupby('name').agg({
    'EX': 'sum',
    'A': 'sum',
    'B': 'sum',
    'C': 'sum',
    'D': 'sum',
    'P': 'sum',
    'F': 'sum',
    'Semester': 'first',  # Take the first semester value
    'L-T-P': 'first',     # Take the first L-T-P value
    'Department Code': 'first'  # Take the first department code value
}).reset_index()

df

,name,EX,A,B,C,D,P,F,Semester,L-T-P,Department Code
0,ADVANCED MASS TRANSFER,1,9,16,13,0,0,0,Spring,4,CH
1,AUDIO SYSTEMS ENGINEERING,38,64,42,9,1,0,0,Spring,3,ET
2,"BIODIVERSITY PROTECTION, FARMERS AND BREEDERS ...",0,0,1,4,1,0,0,Spring,2,IP
3,BOUNDARY INTEGRAL METHODS,0,0,1,0,0,0,0,Spring,4,MA
4,COAL PREPARATION,2,2,21,5,1,0,0,Spring,3,MI
...,...,...,...,...,...,...,...,...,...,...,...
77,SUPRAMOLECULAR CHEMISTRY,0,1,0,0,0,0,0,Spring,3,CY
78,SWITCHING & FINITE AUTOMATA,6,40,61,13,3,0,0,Spring,4,MA
79,TRADE SECRET AND TECHNOLOGY TRANSFER,0,0,0,0,1,0,0,Spring,4,IP
80,TRANSFER AND MANAGEMENT OF RURAL TECHNOLOGY,2,23,20,7,4,1,0,Autumn,4,RD


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Suppose your dataframe is named 'df'

# Vectorize 'Name' using TF-IDF
tfidf_name = TfidfVectorizer()
name_vectors = tfidf_name.fit_transform(df['name'])

# Vectorize 'Department Name' using TF-IDF
tfidf_dept = TfidfVectorizer()
dept_vectors = tfidf_dept.fit_transform(df['Department Code'])

# Create dataframes for the vectorized features
name_df = pd.DataFrame(name_vectors.toarray(), columns=tfidf_name.get_feature_names_out())
dept_df = pd.DataFrame(dept_vectors.toarray(), columns=tfidf_dept.get_feature_names_out())

# Concatenate with the unchanged columns
df_processed = pd.concat([df[['EX', 'A', 'B', 'C', 'D', 'P', 'F', 'Semester']], name_df, dept_df], axis=1)

df_processed 


,EX,A,B,C,D,P,F,Semester,advanced,ageing,...,hs,im,ip,ma,mi,ms,mt,pe,rd,rt
0,1,9,16,13,0,0,0,Spring,0.605431,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,38,64,42,9,1,0,0,Spring,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,4,1,0,0,Spring,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,0,0,0,0,Spring,0.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,2,21,5,1,0,0,Spring,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0,1,0,0,0,0,0,Spring,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,6,40,61,13,3,0,0,Spring,0.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0,0,0,0,1,0,0,Spring,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,2,23,20,7,4,1,0,Autumn,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
df_vec = pd.get_dummies(
df_processed, columns=['Semester'])
df

,name,EX,A,B,C,D,P,F,Semester,L-T-P,Department Code
0,ADVANCED MASS TRANSFER,1,9,16,13,0,0,0,Spring,4,CH
1,AUDIO SYSTEMS ENGINEERING,38,64,42,9,1,0,0,Spring,3,ET
2,"BIODIVERSITY PROTECTION, FARMERS AND BREEDERS ...",0,0,1,4,1,0,0,Spring,2,IP
3,BOUNDARY INTEGRAL METHODS,0,0,1,0,0,0,0,Spring,4,MA
4,COAL PREPARATION,2,2,21,5,1,0,0,Spring,3,MI
...,...,...,...,...,...,...,...,...,...,...,...
77,SUPRAMOLECULAR CHEMISTRY,0,1,0,0,0,0,0,Spring,3,CY
78,SWITCHING & FINITE AUTOMATA,6,40,61,13,3,0,0,Spring,4,MA
79,TRADE SECRET AND TECHNOLOGY TRANSFER,0,0,0,0,1,0,0,Spring,4,IP
80,TRANSFER AND MANAGEMENT OF RURAL TECHNOLOGY,2,23,20,7,4,1,0,Autumn,4,RD


In [36]:
df.iloc[:40,0]

0                                ADVANCED MASS TRANSFER
1                             AUDIO SYSTEMS ENGINEERING
2     BIODIVERSITY PROTECTION, FARMERS AND BREEDERS ...
3                             BOUNDARY INTEGRAL METHODS
4                                      COAL PREPARATION
5                                      COMPLEX ANALYSIS
6                          COMPUTATIONAL LINEAR ALGEBRA
7     COMPUTATIONAL MODELING OF MULTIPHASE REACTIVE ...
8                       COMPUTER METHODS IN CHEM. ENGG.
9                                     COMPUTER NETWORKS
10                         DATA STRUCTURE AND ALGORITHM
11                        ECONOMICS OF ENTREPRENEURSHIP
12                           EDUCATIONAL DATA ANALYTICS
13                             EDUCATIONAL NEUROSCIENCE
14                             ENERGY SYSTEMS MODELLING
15                           ENGINEERING DESIGN PROCESS
16                             ENVIRONMENTAL HUMANITIES
17                              ENVIRONMENTAL SO

In [8]:
min_grade_EX = df_vec['EX'].min()
max_grade_EX = df_vec['EX'].max()
df_vec['EX'] = (df_vec['EX'] - min_grade_EX) / (max_grade_EX - min_grade_EX)

min_grade_A = df_vec['A'].min()
max_grade_A = df_vec['A'].max()
df_vec['A'] = (df_vec['A'] - min_grade_A) / (max_grade_A - min_grade_A)

min_grade_B = df_vec['B'].min()
max_grade_B = df_vec['B'].max()
df_vec['B'] = (df_vec['B'] - min_grade_B) / (max_grade_B - min_grade_B)

min_grade_C = df_vec['C'].min()
max_grade_C = df_vec['C'].max()
df_vec['C'] = (df_vec['C'] - min_grade_C) / (max_grade_C - min_grade_C)

min_grade_D = df_vec['D'].min()
max_grade_D = df_vec['D'].max()
df_vec['D'] = (df_vec['D'] - min_grade_D) / (max_grade_D - min_grade_D)

min_grade_P = df_vec['P'].min()
max_grade_P = df_vec['P'].max()
df_vec['P'] = (df_vec['P'] - min_grade_P) / (max_grade_P - min_grade_P)

min_grade_F = df_vec['F'].min()
max_grade_F = df_vec['F'].max()
df_vec['F'] = (df_vec['F'] - min_grade_F) / (max_grade_F - min_grade_F)

In [9]:
df_vec

,EX,A,B,C,D,P,F,advanced,ageing,algebra,...,ip,ma,mi,ms,mt,pe,rd,rt,Semester_Autumn,Semester_Spring
0,0.009434,0.062937,0.108108,0.089041,0.000000,0.000000,0.0,0.605431,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
1,0.358491,0.447552,0.283784,0.061644,0.009434,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
2,0.000000,0.000000,0.006757,0.027397,0.009434,0.000000,0.0,0.000000,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
3,0.000000,0.000000,0.006757,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
4,0.018868,0.013986,0.141892,0.034247,0.009434,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.000000,0.006993,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
78,0.056604,0.279720,0.412162,0.089041,0.028302,0.000000,0.0,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
79,0.000000,0.000000,0.000000,0.000000,0.009434,0.000000,0.0,0.000000,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True
80,0.018868,0.160839,0.135135,0.047945,0.037736,0.023256,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,True,False


In [25]:
df_vec.columns[-20:]

Index(['well', 'bm', 'ch', 'cr', 'cy', 'ep', 'es', 'et', 'hs', 'im', 'ip',
       'ma', 'mi', 'ms', 'mt', 'pe', 'rd', 'rt', 'Semester_Autumn',
       'Semester_Spring'],
      dtype='object')

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

department_similarity = cosine_similarity(df_vec[['bm', 'ch', 'cr', 'cy', 'ep', 'es', 'et', 'hs', 'im', 'ip',
       'ma', 'mi', 'ms', 'mt', 'pe', 'rd', 'rt',]])

course_similarity = cosine_similarity(df_vec.drop(['Semester_Spring', 'Semester_Autumn','EX', 'A', 'B', 'C', 'D', 'P', 'F', 'bm', 'ch', 'cr', 'cy', 'ep', 'es', 'et', 'hs', 'im', 'ip',
       'ma', 'mi', 'ms', 'mt', 'pe', 'rd', 'rt',], axis=1))

# Combine the similarities
final_similarity = 0.5 * department_similarity +  0.5*course_similarity

print(final_similarity)

[[1.         0.         0.         ... 0.11711521 0.11881113 0.5       ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.53171416 0.03217341 0.        ]
 ...
 [0.11711521 0.         0.53171416 ... 1.         0.22085874 0.        ]
 [0.11881113 0.         0.03217341 ... 0.22085874 1.         0.        ]
 [0.5        0.         0.         ... 0.         0.         1.        ]]


In [10]:
# from sklearn.metrics.pairwise import cosine_similarity


# vectorized_columns = df_vec.drop(['Semester_Spring', 'Semester_Autumn','EX', 'A', 'B', 'C', 'D', 'P', 'F'],axis=1)
# vectorized_similarity = cosine_similarity(vectorized_columns)

# one_hot_encoded_columns = df_vec[['Semester_Spring', 'Semester_Autumn']]
# one_hot_encoded_similarity = cosine_similarity(one_hot_encoded_columns)

# final_similarity = vectorized_similarity + 0.3*one_hot_encoded_similarity 


In [42]:
final_similarity

array([[1.        , 0.        , 0.        , ..., 0.21080737, 0.21386003,
        0.1       ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.15708549, 0.05791213,
        0.        ],
       ...,
       [0.21080737, 0.        , 0.15708549, ..., 1.        , 0.39754572,
        0.        ],
       [0.21386003, 0.        , 0.05791213, ..., 0.39754572, 1.        ,
        0.        ],
       [0.1       , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [48]:
def generate_recommendations(course_name, similarity_matrix, df, top_n=5):
    course_index = df[df['name'] == course_name].index[0]
    
    course_similarity = similarity_matrix[course_index]
    
    similar_courses_indices = course_similarity.argsort()[::-1][1:top_n+1]  
    similar_courses = df.iloc[similar_courses_indices]['name'].values
    
    return similar_courses

recommendations = generate_recommendations('DATA STRUCTURE AND ALGORITHM', final_similarity, df)
print("Recommendations based on similarity:")
print(recommendations)

Recommendations based on similarity:
['STOCHASTIC PROCESS AND SIMULATION' 'MEASURE THEORY AND INTEGRATION'
 'INTEGRAL EQUATIONS AND VARIATIONAL METHODS' 'COMPUTER NETWORKS'
 'REGRESSION & TIME SERIES MODELS']


In [49]:
df[df['name']=='STOCHASTIC PROCESS AND SIMULATION']

,name,EX,A,B,C,D,P,F,Semester,L-T-P,Department Code
74,STOCHASTIC PROCESS AND SIMULATION,4,7,9,8,1,0,0,Spring,4,MA


In [50]:
import pickle 

pickle.dump(df,open('/kaggle/working/df.pkl','wb'))
pickle.dump(final_similarity,open('/kaggle/working/similarity.pkl','wb'))

In [60]:
from fuzzywuzzy import process

def generate_recommendations(course_name, similarity_matrix, df, top_n=5):
    # Perform fuzzy matching to find the closest match to the given course name
    match = process.extractOne(course_name, df['name'])
    closest_course_name = match[0]
    
    # Get the index of the closest course name
    course_index = df[df['name'] == closest_course_name].index[0]
    
    # Calculate similarity based on the index
    course_similarity = similarity_matrix[course_index]
    
    # Get indices of similar courses
    similar_courses_indices = course_similarity.argsort()[::-1][1:top_n+1]  
    similar_courses = df.iloc[similar_courses_indices]['name'].values
    
    return closest_course_name, similar_courses

# Example usage
partial_course_name = 'soil'
closest_match, recommendations = generate_recommendations(partial_course_name, final_similarity, df)
print("Closest match to the partial course name:", closest_match)
print("Recommendations based on similarity:")
print(recommendations)


Closest match to the partial course name: ENVIRONMENTAL SOCIOLOGY
Recommendations based on similarity:
['INTRODUCTION TO SOCIOLOGY' 'ENVIRONMENTAL HUMANITIES'
 'INTRODUCTION TO CULTURAL STUDIES'
 'SOCIO-TECHNICAL INTERVENTIONS FOR AGEING SOCIETIES'
 'INTRODUCTION TO ETHICS']


In [61]:
from fuzzywuzzy import process

def generate_recommendations_by_department(department_code, similarity_matrix, df, top_n=5):
    # Filter the DataFrame to include only courses from the given department code
    department_courses = df[df['Department Code'] == department_code]
    
    # Get the index of the department courses
    department_indices = department_courses.index
    
    # Calculate the average similarity for courses in the department
    department_similarity = similarity_matrix[department_indices].mean(axis=0)
    
    # Get indices of similar courses
    similar_courses_indices = department_similarity.argsort()[::-1][:top_n]  
    similar_courses = df.iloc[similar_courses_indices]['name'].values
    
    return similar_courses

# Example usage
department_code = 'IM'
recommendations = generate_recommendations_by_department(department_code, final_similarity, df)
print("Recommendations based on department code similarity:")
print(recommendations)


Recommendations based on department code similarity:
['LOGISTICS SYSTEMS AND MANAGEMENT' 'NETWORKS AND PROJECT MANAGEMENT'
 'SAFETY ANALYTICS' 'OPERATIONS RESEARCH' 'MARKETING AND MARKET RESEARCH']


In [62]:
def generate_recommendations(course_name, department_code, similarity_matrix, df, top_n=5):
    if course_name:
        recommendations = generate_recommendations_by_course(course_name, similarity_matrix, df, top_n)
    elif department_code:
        recommendations = generate_recommendations_by_department(department_code, similarity_matrix, df, top_n)
    else:
        recommendations = []
    return recommendations

def generate_recommendations_by_course(course_name, similarity_matrix, df, top_n=5):
    closest_match, _ = process.extractOne(course_name, df['name'])
    return generate_recommendations(closest_match, similarity_matrix, df, top_n)

def generate_recommendations_by_department(department_code, similarity_matrix, df, top_n=5):
    department_courses = df[df['Department Code'] == department_code]
    department_indices = department_courses.index
    department_similarity = similarity_matrix[department_indices].mean(axis=0)
    similar_courses_indices = department_similarity.argsort()[::-1][:top_n]  
    similar_courses = df.iloc[similar_courses_indices]['name'].values
    return similar_courses

# Example usage
course_name = 'DATA STRUCTURE AND ALGORITHM'
department_code = None
recommendations = generate_recommendations(course_name, department_code, final_similarity, df)
print("Recommendations based on course name similarity:")
print(recommendations)

course_name = None
department_code = 'MA'
recommendations = generate_recommendations(course_name, department_code, final_similarity, df)
print("Recommendations based on department code similarity:")
print(recommendations)


ValueError: too many values to unpack (expected 2)